# All methods of using tests

There are some things you can and some things you can't do with tests.  This tutorial will show all of them.

In [1]:
from nornir_tests.plugins.processors import TestsProcessor
from nornir_tests.plugins.tests import *
from nornir_napalm.plugins.tasks import napalm_get, napalm_ping
from nornir_rich.plugins.functions import RichResults
from nornir import InitNornir

nr = InitNornir(
    inventory={
        "plugin": "SimpleInventory",
        "options": {
            "host_file": "data/hosts.yaml",
            "group_file": "data/groups.yaml",
            "defaults_file": "data/defaults.yaml",
        },
    },
)

rr = RichResults()
nr.processors.append(TestsProcessor())

vyos = nr.filter(name="vyos")

## @ style decorator use

This will only work as designed if not using task.run inside the function.  If there were a bunch of task.run statements in the function the decorators applied using @ syntax would not get applied.

In [2]:
@test_until(retries=5, delay=5)
@test_timing(max_run_time=5, fail_task=True)
@test_jsonpath(path='interfaces.eth0.is_enabled', assertion='is_true', fail_task=True)
def at_syntax_test(task):
    return napalm_get(task, getters=['interfaces'])

rr.print(vyos.run(task=at_syntax_test))

## Using nr.run with tasks that return results

In [3]:
rr.print(vyos.run(
    task=napalm_get,
    getters=['interfaces'],
    tests=[
        test_timing(),
        test_jsonpath(path='interfaces.eth0.is_up', assertion='is_true')
    ]
))

## Using task.run inside grouped task

In [4]:
def grouped_task(task):
    task.run(
        napalm_ping,
        dest='192.168.99.1',
        tests=[
            test_timing()
        ]
    )

    task.run(
        napalm_get, 
        getters=['interfaces'], 
        tests=[
            test_jsonpath(path='interfaces.eth0.is_up', assertion='is_true')
        ]
    )

rr.print(vyos.run(task=grouped_task), vars=['tests'])

## Not supported

Testing a grouped_task at the level of running nr.run will not work.  There is not much you would be able to test without really complex logic anyhow considering this always returns an AggregatedResult.